# This is a step by step approach to Lab 08
## All Code provided here is sample code - if you just copy and paste it, it may not work
## If you have any doubts, feel free to reach out to me at vzm5268@psu.edu

In [ ]:
from datascience import *
path_data = "https://raw.githubusercontent.com/DS200-SP2024-Hunter/Lab08-DueMar20/main/FiveKResults.csv"
import numpy as np

import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

# Task 1:
Load the dataset (the URL is given above) into python. Give it a sensible name that you can remember, like fiveK.



In [ ]:
fiveK = Table.read_table(path_data)

In [ ]:
fiveK.show(5)

Run,Finishing Time In Minutes,Identifies As Female,Age,Name,City,State
AF,17.617,False,27,Andy Cunningham,State College,PA
AF,18.067,False,49,S Mark Courtney,Grove City,PA
AF,18.583,False,46,Terry Reid,Bellefonte,PA
AF,18.717,False,15,Justin Rupert,Wellsboro,PA
AF,18.8,True,35,Kate Fonshell,Ardmore,PA


# Task 2
Perform an A/B test of the null hypothesis that the hypothetical average male time in this race equals the hypothetical average female time in this race, among all 5K runners. For this purpose, you can either use the difference_of_means function from Section 12.1 or you can create your own version of this function using pandas.



In [ ]:
# define your NULL Hypothesis
# define your test statistics

In [ ]:
'''We are going compute such differences repeatedly in our simulations below, so we will define a function to do the job. The function takes two arguments:
      the name of the table of data
      the label of the column that contains the Boolean variable for grouping
      It returns the difference between the means of the True group and the False group'''

# Purposefully I have not modified the sample code according to our scenario
# It is your job to modify it as needed

def difference_of_means(table, group_label):
    reduced = table.select('Birth Weight', group_label)
    means_table = reduced.group(group_label, np.average)
    means = means_table.column(1)
    return means.item(1) - means.item(0)

In [ ]:
'''If there were no difference between the two distributions in the underlying population, then whether Identifies as Female has the label True or False with respect to
Finishing Time should make no difference to the average. The idea, then, is to shuffle all the labels randomly among the participants. This is called random permutation.
'''

# Purposefully I have not modified the sample code according to our scenario
# It is your job to modify it as needed

def one_simulated_difference_of_means():
    """Returns: Difference between mean birthweights
    of babies of smokers and non-smokers after shuffling labels"""

    # array of shuffled labels
    shuffled_labels = births.sample(with_replacement=False).column('Maternal Smoker')

    # table of birth weights and shuffled labels
    shuffled_table = births.select('Birth Weight').with_column(
        'Shuffled Label', shuffled_labels)

    return difference_of_means(shuffled_table, 'Shuffled Label')

In [ ]:
# Perform this task for many repetitions
# Notice: In this version, we set up a full array with the correct number of entries as a first step.
repetitions = 5000
differences = np.empty(repetitions)

for i in np.arange(repetitions):
    differences[i] = one_simulated_difference_of_means()

# Task 3
Plot the histogram of mean differences that you obtain by simulation under the null hypothesis of no mean difference. Compare this histogram to the value observed in the sample, calculate a p-value, and comment on what this p-value tells you in this case about whether we should reject the null hypothesis. Write a conclusion sentence that conveys what you have discovered to an audience unfamiliar with statistical terminology.



In [ ]:
Table().with_column('Difference Between Group Means', differences).hist()
print('Observed Difference:', observed_difference)
plots.title('Prediction Under the Null Hypothesis')

In [ ]:
# observed value corresponds to the original mean difference before NULL hypothesis is performed
# refer to section  12.1 from textbook if needed
empirical_p = np.count_nonzero(differences <= observed_difference) / repetitions
empirical_p

# Task 4

Construct a confidence interval for the true mean difference (male mean minus female mean) using bootstrapping. Just as in Section 13.2, it will be possible to use the sample() method with no arguments to accomplish the bootstrapping. However, there is a subtle point in this case: We should make sure that each bootstrap sample has the same number of males and females as the original, since we're finding the difference of two sample means (one female, one male) and the sample sizes are important in determining the behavior of sample means. Therefore, if you're using datascience then you'll want to create two separate Table objects, one for females and one for males, using code such as this:

In [ ]:
fiveK_Male = fiveK.where('Identifies As Female', False)
fiveK_Female = fiveK.where('Identifies As Female', True)

# Task 5

Rewrite the function one_bootstrap_median in Section 13.2.6. so that it selects one bootstrap sample for females, another for males, then returns the difference of the two mean finishing times. Here is code that accomplishes this if you're using the datascience package:

In [ ]:
# Now perform Bootstripping on fiveK_Male and fiveK_Female and find the mean difference
def one_bootstrap_mean_difference():
    resampled_Females = fiveK_Female.sample()
    resampled_Males = fiveK_Male.sample()
    Female_mean = np.mean(resampled_Females.column('Finishing Time In Minutes'))
    Male_mean = np.mean(resampled_Males.column('Finishing Time In Minutes'))
    bootstrapped_mean_difference =  Male_mean - Female_mean
    return bootstrapped_mean_difference

# Task 6
Reproduce the analysis of Section 13.2.7 by creating a histogram of 5000 bootstrapped mean differences. You will have to choose histogram bins that are appropriate for this example AND there will not be a green dot showing the position of the population parameter (since in this case we do not know the population parameter).

In [ ]:
num_repetitions = 5000
bstrap_means = make_array()
for i in np.arange(num_repetitions):
    bstrap_means = np.append (bstrap_means, one_bootstrap_mean_difference())

In [ ]:
resampled_means = Table().with_column('Bootstrap Sample Median', bstrap_means)
# initialize mean beans as per the distribution, try printing the resample_means and see the range
# mean_bins=np.arange(120000, 160000, 2000)
resampled_means.hist(bins = mean_bins)



# Task 7
Find the 2.5 and 97.5 percentiles of your mean difference distribution, and use these values as a 95% confidence interval. You should include a text box in your output that says "We are 95% confident that the true difference in mean 5K road race finishing times between females and males is between ----- and -----." (You will fill in the blanks.)

In [ ]:
left = percentile(2.5, bstrap_means)
right = percentile(97.5, bstrap_means)
print(left, right)

# Task 8
Finally, make sure that your Jupyter notebook only includes code and text that is relevant to this assignment. For instance, if you have been completing this assignment by editing some original code from Section 12.1 or Section 13.2, make sure to delete the material that isn't relevant before turning in your work.

When you've completed this, you should select "Print" from the File menu, then save to pdf using this option.  The pdf file that you create in this way is the file that you should upload to Canvas for grading.  We have found that if you can select the "A3" paper size from the advanced options, this seems to solve the problems that are sometimes encountered in this step